# 2. 의사록 섹션 분리  

 - 괴상한 특수문자(.으로 잡지 못하는 특수문자)를 정규표현식을 이용하여 제거
 - 별도 제작한 정규표현식으로 섹션 분리

In [ ]:
data = pd.read_excel('tika\\금통위의사록(텍스트파일).xlsx', usecols = [1,2])
data.columns = ['text', 'date']
data.head(2)

In [ ]:
import re
for i in range(len(data)):
    data.loc[i, 'text'] = re.sub('- \d+ -', '', data.loc[i, 'text'])
    data.loc[i, 'text'] = re.sub('―|｢|｣|-|[․/→←+]', '', data.loc[i, 'text'])
data.head(2)

### 문장으로 분리

In [ ]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])+[ \s\n\.;\(]+|(?<=다)\.)\s+')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

### 섹션 분리

In [ ]:
def txt_split(data):
    result_list = []
    for i in range(len(data)):
        time = data.loc[i, 'date']
        text = data.loc[i, 'text']

        p1 = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)|｢?.?외환.?국제금융\s?동향.?｣?과 관련하여.*' , text)
        p2 = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은|｢?.?금융시장\s?동향.?｣?과 관련하여,?', text)
        p3 = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', text)
        
        s1 = p1.start() if p1 else -1
        s2 = p2.start() if p2 else -1
        s3 = p3.start() if p2 else -1
        
        section_2 = text[s1:s2] if s1 >= 0 or s2 >= 0 else ''
        section_3 = text[s2:s3] if s2 >= 0 or s3 >= 0 else ''
        
        sentences2, text2 = tidy_sentences(section_2)
        sentences3, text3 = tidy_sentences(section_3)
        new_dict = {
            'date' : time,
            's2' : text2,
            's3' : text3
        }
        result_list.append(new_dict)
        
    df = pd.DataFrame(result_list)
    return df